In [1]:
import rasa_nlu
import rasa_core
import spacy

In [3]:
print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("en")("Hello world!"))

rasa_nlu: 0.14.6 rasa_core: 0.9.0
Loading spaCy language model...
Hello world!


In [19]:
nlu_md = """
## intent:greet
- hey
- hello
- good morning
- good evening
- howdy

## intent:goodbye
- goodbye
- see you
- good by
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [20]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_spacy"                   # spacy component that uses pretrained entities
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


# Training  Rasa NLU Model

In [32]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
model = trainer.train(training_data, )

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:113

In [31]:
import pprint as pp
pp.pprint(interpreter.parse("doing great"))

{'entities': [],
 'intent': {'confidence': 0.6519458459360967, 'name': 'greet'},
 'intent_ranking': [{'confidence': 0.6519458459360967, 'name': 'greet'},
                    {'confidence': 0.3480541540639033, 'name': 'goodbye'}],
 'text': 'doing great'}


In [4]:
stories_md = """

"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


In [5]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy

actions:
- utter_greet
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- __main__.

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up: summer is around the corner!"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


In [7]:
from IPython.display import Image
from rasa_core.agent import Agent

In [18]:
agent = Agent('domain.yml')

# agent.visualize("stories.md", "story_graph.png", max_history=2)
# Image(filename="story_graph.png")

ValueError: Action '__main__.' doesn't correspond to a template / action. Module doesn't contain a class with this name. Remember to prefix actions that should utter a template with `utter_`. Error: module '__main__' has no attribute ''

In [10]:

from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=400
)

agent.persist('models/dialogue')

Using TensorFlow backend.


ValueError: Action '__main__.' doesn't correspond to a template / action. Module doesn't contain a class with this name. Remember to prefix actions that should utter a template with `utter_`. Error: module '__main__' has no attribute ''